In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold, cross_validate, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
raw_data = pd.read_excel('data/Copy of clean.xlsx', sheet_name='clean')

# Preprocessing

In [3]:
X = raw_data.drop(['WIWO'], axis=1)
y = raw_data['WIWO']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=101, test_size=0.1)

# Transformation

## Mapping and other constant variables

In [5]:
DOW_MAPPING = ({
    '月': 1,
    '火': 2,
    '水': 3,
    '木': 4,
    '金': 5,
    '土': 6,
    '日': 7,
})

In [6]:
UNIQUE_ID = ['surgery_id']

In [7]:
INITIAL_FEATURES = [
    'dow', 
    'surgery_classification_name',
    'surgery_department_code',
    'surgery_diagnosis_code', 
    'surgery_diagnosis_code2', 
    'surgery_diagnosis_code3', 
    'surgery_diagnosis_code4', 
    'surgery_diagnosis_code5',
    'surgery_internal_code',
    'surgery_internal_code2',
    'surgery_internal_code3',
    'surgery_internal_code4',
    'surgery_anes_method_code',
    'or_vendor_code',
    # 'or_reservation_entry_time',
    'or_reservation_operation_duration',
    'staff_main_surgeon_name', 
    'staff_main_surgeon_name2',
    'staff_main_surgeon_name3', 
    'staff_main_surgeon_name4',
    'staff_main_surgeon_name5', 
    'staff_main_surgeon_name6',
    'staff_main_surgeon_name7', 
    'staff_main_surgeon_name8',
    'surgeon_anes_name1', 
    'surgeon_anes_name2', 
    'surgeon_anes_name3',
    'surgeon_anes_name4', 
    'surgeon_anes_name5', 
    'scrub_nurse_name1',
    'scrub_nurse_name2', 
    'scrub_nurse_name3', 
    'scrub_nurse_name4',
    'circulating_nurse_name1', 
    'circulating_nurse_name2',
    'circulating_nurse_name3', 
    'circulating_nurse_name4',
    'surgery_ward_code', 
    'surgery_position_code2',
    'surgery_position_name',
    'surgery_equipment_vendor_code1',
    'surgery_equipment_vendor_code2',
    'surgery_equipment_vendor_code3',
    'surgery_equipment_vendor_code4',
    'surgery_equipment_vendor_code5',
    'surgery_equipment_vendor_code6',
    'surgery_equipment_vendor_code7',
    'surgery_equipment_vendor_code8',
    'on_call', 'cancel',
    'wom', 'month', 'year',
]

In [8]:
SURGERY_DIAGNOSIS_CODES = [
    'surgery_diagnosis_code', 
    'surgery_diagnosis_code2', 
    'surgery_diagnosis_code3', 
    'surgery_diagnosis_code4', 
    'surgery_diagnosis_code5',
] 

In [9]:
SURGERY_INTERNAL_CODES = [
    'surgery_internal_code',
    'surgery_internal_code2',
    'surgery_internal_code3',
    'surgery_internal_code4',
]

In [10]:
STAFF_MAIN_SURGEON_NAMES = [
    'staff_main_surgeon_name', 
    'staff_main_surgeon_name2',
    'staff_main_surgeon_name3', 
    'staff_main_surgeon_name4',
    'staff_main_surgeon_name5', 
    'staff_main_surgeon_name6',
    'staff_main_surgeon_name7', 
    'staff_main_surgeon_name8',
]

In [11]:
SURGEON_ANES_NAMES = [
    'surgeon_anes_name1', 
    'surgeon_anes_name2', 
    'surgeon_anes_name3',
    'surgeon_anes_name4', 
    'surgeon_anes_name5',
]

In [12]:
SCRUB_NURSE_NAMES = [
    'scrub_nurse_name1',
    'scrub_nurse_name2', 
    'scrub_nurse_name3', 
    'scrub_nurse_name4',
]

In [13]:
CIRCULATING_NURSE_NAMES = [
    'circulating_nurse_name1', 
    'circulating_nurse_name2',
    'circulating_nurse_name3', 
    'circulating_nurse_name4',
]

In [14]:
SURGERY_EQUIPMENT_VENDOR_CODES = [
    'surgery_equipment_vendor_code1',
    'surgery_equipment_vendor_code2',
    'surgery_equipment_vendor_code3',
    'surgery_equipment_vendor_code4',
    'surgery_equipment_vendor_code5',
    'surgery_equipment_vendor_code6',
    'surgery_equipment_vendor_code7',
    'surgery_equipment_vendor_code8',
]

In [15]:
CATEGORICAL_FEATURES = [
#     'dow',
    'surgery_classification_name',
    'surgery_department_code',
    'surgery_diagnosis_code', 
    'surgery_internal_code',
    'surgery_anes_method_code',
    'or_vendor_code',
    'staff_main_surgeon_name', 
    'surgeon_anes_name1', 
    'surgeon_anes_name2', 
    'surgeon_anes_name3',
    'surgeon_anes_name4', 
    'surgeon_anes_name5', 
    'scrub_nurse_name1',
    'scrub_nurse_name2', 
    'circulating_nurse_name1', 
    'circulating_nurse_name2',
    'surgery_ward_code', 
    'surgery_position_code2',
    'surgery_position_name',
    'surgery_equipment_vendor_code1',
    'surgery_equipment_vendor_code2',
    'on_call', 'cancel',
    'wom', 'month', 'year',
]

In [16]:
NUMERICAL_FEATURES = [
    'or_reservation_operation_duration',
]

In [17]:
TRANSFORMED_FEATURES = [
    'transformed_dow'
]

## Transformers

In [18]:
class CountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        count_non_na = pd.DataFrame(X.count(axis=1))
        return count_non_na

In [19]:
class AddDowTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Expand original X with new columns
        X = X.copy()
        X.loc[:, 'transformed_dow'] = X['dow'].map(DOW_MAPPING)
        return X

In [20]:
class FeatureSelectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.features]

In [21]:
# class ImpactEncodingTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, feature_name):
#         self.feature_name = feature_name
#         self.X_and_y = pd.DataFrame()
#         self.impact_mapping = dict()
#         self.impact_encoded_feature = pd.Series(dtype=int)
    
#     def fit(self, X, y):
#         self.X_and_y = pd.concat([X, y], axis=1)
#         self.impact_mapping = self.X_and_y.groupby(self.feature_name)['WIWO'].mean().astype(int).to_dict()
#         return self.impact_mapping
    
#     def transform(self, X, y=None):
#         self.impact_encoded_feature = X[self.feature_name].map(self.impact_mapping)
#         return self.impact_encoded_feature

In [22]:
class ImpactEncodingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.X_and_y = pd.DataFrame()
        self.feature_names = list()
        self.y_name = str()
        self.y_mean = int()
        self.impact_mapping = dict()
        # self.impact_encoded_features creates a bug where .predict doesn't work in a pipeline
    
    def fit(self, X, y):
        # Setting up variables during fit
        self.X_and_y = pd.concat([X, y], axis=1)
        self.feature_names = list(pd.DataFrame(X).columns)
        self.y_name = y.name
        self.y_mean = int(y.mean())
        
        # Getting impact encoding mapping for all features
        for feature_name in self.feature_names:
            self.impact_mapping[feature_name] = \
                self.X_and_y.groupby(feature_name)[self.y_name].mean().astype(int).to_dict()
        
        # Must return self as a transformer, otherwise errors
        return self
    
    def transform(self, X, y=None):
        # Encode all features
        impact_encoded_features = pd.DataFrame()
        for feature_name in self.feature_names:
            # For unknown keys, fill in with global mean
            current_encoded_feature = \
                X[feature_name].map(self.impact_mapping[feature_name]).fillna(self.y_mean).astype(int)
            impact_encoded_features = \
                pd.concat([impact_encoded_features, current_encoded_feature], axis=1)
    
        return impact_encoded_features

In [23]:
combine_all_transformers = make_column_transformer(
    (ImpactEncodingTransformer(), TRANSFORMED_FEATURES + CATEGORICAL_FEATURES),
    (CountTransformer(), SURGERY_DIAGNOSIS_CODES),
    (CountTransformer(), SURGERY_INTERNAL_CODES),
    (CountTransformer(), STAFF_MAIN_SURGEON_NAMES),
    (CountTransformer(), SURGEON_ANES_NAMES),
    (CountTransformer(), SCRUB_NURSE_NAMES),
    (CountTransformer(), CIRCULATING_NURSE_NAMES),
    (CountTransformer(), SURGERY_EQUIPMENT_VENDOR_CODES),
    ('passthrough', NUMERICAL_FEATURES)
)

# Model in a Pipeline

In [24]:
pipe_lm = Pipeline(
    steps=[
        ('select_initial_features', FeatureSelectorTransformer(INITIAL_FEATURES)),
        ('add_dow_transformer', AddDowTransformer()),
        ('select_more_features', FeatureSelectorTransformer(TRANSFORMED_FEATURES + INITIAL_FEATURES)),
        ('combine_all_transformers', combine_all_transformers),
        ('standard_scaler', StandardScaler()),
        ('linear_regression', LinearRegression()),
    ]
)

In [25]:
pipe_lm.fit(X_train, y_train)

Pipeline(steps=[('select_initial_features',
                 FeatureSelectorTransformer(features=['dow',
                                                      'surgery_classification_name',
                                                      'surgery_department_code',
                                                      'surgery_diagnosis_code',
                                                      'surgery_diagnosis_code2',
                                                      'surgery_diagnosis_code3',
                                                      'surgery_diagnosis_code4',
                                                      'surgery_diagnosis_code5',
                                                      'surgery_internal_code',
                                                      'surgery_internal_code2',
                                                      'surgery_internal_code3',
                                                      'surg...
                                                   'surgery_equipment_vendor_code3',
                                                   'surgery_equipment_vendor_code4',
                                                   'surgery_equipment_vendor_code5',
                                                   'surgery_equipment_vendor_code6',
                                                   'surgery_equipment_vendor_code7',
                                                   'surgery_equipment_vendor_code8']),
                                                 ('passthrough', 'passthrough',
                                                  ['or_reservation_operation_duration'])])),
                ('standard_scaler', StandardScaler()),
                ('linear_regression', LinearRegression())])

In [26]:
pipe_lm.predict(X_test)

array([ 57.72471069, 144.53279824, 279.0487039 , ..., 554.37888338,
       326.94809067,  79.04809534])

In [27]:
pipe_xgboost = Pipeline(
    steps=[
        ('select_initial_features', FeatureSelectorTransformer(INITIAL_FEATURES)),
        ('add_dow_transformer', AddDowTransformer()),
        ('select_more_features', FeatureSelectorTransformer(TRANSFORMED_FEATURES + INITIAL_FEATURES)),
        ('combine_all_transformers', combine_all_transformers),
        ('standard_scaler', StandardScaler()),
        ('xgboost', XGBRegressor()),
    ]
)

In [28]:
pipe_xgboost.fit(X_train, y_train)

Pipeline(steps=[('select_initial_features',
                 FeatureSelectorTransformer(features=['dow',
                                                      'surgery_classification_name',
                                                      'surgery_department_code',
                                                      'surgery_diagnosis_code',
                                                      'surgery_diagnosis_code2',
                                                      'surgery_diagnosis_code3',
                                                      'surgery_diagnosis_code4',
                                                      'surgery_diagnosis_code5',
                                                      'surgery_internal_code',
                                                      'surgery_internal_code2',
                                                      'surgery_internal_code3',
                                                      'surg...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [29]:
pipe_xgboost.predict(X_test)

array([ 41.463104, 153.98106 , 283.5324  , ..., 550.92    , 335.8193  ,
        60.97048 ], dtype=float32)

# Cross Validation

In [30]:
def cross_validated_results(model_results):
    df = pd.DataFrame()
    df['MAE'] = model_results['test_neg_mean_absolute_error']
    df['R2'] = model_results['test_r2']
    return df.describe()

In [31]:
cv = KFold(n_splits=5, shuffle=True, random_state=101)
scoring = ['neg_mean_absolute_error', 'r2']

In [32]:
lm_results = cross_validate(pipe_lm, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)

In [33]:
cross_validated_results(lm_results)

,MAE,R2
count,5.000000,5.000000
mean,-42.662376,0.865475
std,0.563866,0.012391
min,-43.419203,0.850640
25%,-42.822991,0.859305
50%,-42.628487,0.862045
75%,-42.595909,0.872907
max,-41.845290,0.882475


In [34]:
xgboost_results = cross_validate(pipe_xgboost, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)

In [35]:
cross_validated_results(xgboost_results)

,MAE,R2
count,5.000000,5.000000
mean,-39.286703,0.872470
std,0.991554,0.011781
min,-40.569785,0.853337
25%,-39.589089,0.871815
50%,-39.481999,0.873431
75%,-38.930770,0.879590
max,-37.861870,0.884178


# Fine tuning

In [36]:
"""
5 iters with cv=3:
{'xgboost__subsample': 0.8,
 'xgboost__reg_alpha': 0.01,
 'xgboost__min_child_weight': 5,
 'xgboost__max_depth': 3,
 'xgboost__gamma': 0.3,
 'xgboost__colsample_bytree': 0.9}

100 iters with cv=3:
{'xgboost__subsample': 0.8,
 'xgboost__reg_alpha': 100,
 'xgboost__min_child_weight': 3,
 'xgboost__max_depth': 3,
 'xgboost__gamma': 0.1,
 'xgboost__colsample_bytree': 0.7}
 
400 iters with cv=5:
{'xgboost__subsample': 1.0,
 'xgboost__reg_alpha': 0.01,
 'xgboost__min_child_weight': 1,
 'xgboost__max_depth': 5,
 'xgboost__gamma': 0.7,
 'xgboost__colsample_bytree': 0.3}

1000 iters with cv=5
{'xgboost__subsample': 1.0,
 'xgboost__reg_alpha': 0.5,
 'xgboost__min_child_weight': 1,
 'xgboost__max_depth': 5,
 'xgboost__gamma': 0.0,
 'xgboost__colsample_bytree': 0.4}
""";

In [37]:
xgboost = XGBRegressor(
     subsample=1.0,
     reg_alpha=0.5,
     min_child_weight=1,
     max_depth=5,
     gamma=0.0,
     colsample_bytree=0.4,
     random_state=4,
     n_jobs=-1
)

In [38]:
tuned_xgboost = Pipeline(
    steps=[
        ('select_initial_features', FeatureSelectorTransformer(INITIAL_FEATURES)),
        ('add_dow_transformer', AddDowTransformer()),
        ('select_more_features', FeatureSelectorTransformer(TRANSFORMED_FEATURES + INITIAL_FEATURES)),
        ('combine_all_transformers', combine_all_transformers),
        ('standard_scaler', StandardScaler()),
        ('xgboost', xgboost),
    ]
)

In [39]:
tuned_xgboost.fit(X_train, y_train)

Pipeline(steps=[('select_initial_features',
                 FeatureSelectorTransformer(features=['dow',
                                                      'surgery_classification_name',
                                                      'surgery_department_code',
                                                      'surgery_diagnosis_code',
                                                      'surgery_diagnosis_code2',
                                                      'surgery_diagnosis_code3',
                                                      'surgery_diagnosis_code4',
                                                      'surgery_diagnosis_code5',
                                                      'surgery_internal_code',
                                                      'surgery_internal_code2',
                                                      'surgery_internal_code3',
                                                      'surg...
                              feature_types=None, gamma=0.0, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None, min_child_weight=1,
                              missing=nan, monotone_constraints=None,
                              n_estimators=100, n_jobs=-1,
                              num_parallel_tree=None, predictor=None,
                              random_state=4, ...))])

In [40]:
model_params =  {
    'xgboost__max_depth':[1,3,5,8],
    'xgboost__min_child_weight':range(1,6,1),
    'xgboost__gamma':[i/10.0 for i in range(0,10)],
    'xgboost__subsample':[i/10.0 for i in range(3,15)],
    'xgboost__colsample_bytree':[i/10.0 for i in range(1,10)],
    'xgboost__reg_alpha':[1e-5, 1e-2, 0.03, 0.05, 0.08, 0.1, 0.15, 0.2, 0.5, 1, 100],
}

In [41]:
# tuned_xgboost = RandomizedSearchCV(tuned_xgboost, param_distributions=model_params, n_iter=1000, cv=5, random_state=42, n_jobs=-1)

In [42]:
# tuned_xgboost.get_params().keys()

In [43]:
# tuned_xgboost.fit(X_train, y_train)

In [44]:
# tuned_xgboost.best_params_

In [45]:
tuned_xgboost_results = cross_validate(tuned_xgboost, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)

In [46]:
cross_validated_results(tuned_xgboost_results)

,MAE,R2
count,5.000000,5.000000
mean,-38.886574,0.876881
std,0.798537,0.010772
min,-40.031080,0.861428
25%,-39.391325,0.872857
50%,-38.536702,0.876112
75%,-38.335282,0.885647
max,-38.138478,0.888359


# Final test set validation

In [47]:
lm_model = pipe_lm.fit(X_train, y_train)

In [48]:
xgboost_model = pipe_xgboost.fit(X_train, y_train)

In [49]:
tuned_xgboost_model = tuned_xgboost.fit(X_train, y_train)

In [50]:
def test_performance(model, X_test, y_test):
    scores = {}
    predictions = model.predict(X_test)
    scores['mae'] = mean_absolute_error(y_test, predictions)
    scores['r2'] = r2_score(y_test, predictions)
    return scores

In [51]:
test_performance(lm_model, X_test, y_test)

{'mae': 39.7884165164802, 'r2': 0.8870015433693806}

In [52]:
test_performance(xgboost_model, X_test, y_test)

{'mae': 36.73000975700348, 'r2': 0.8931875130422338}

In [53]:
test_performance(tuned_xgboost_model, X_test, y_test)

{'mae': 35.10664303610096, 'r2': 0.903455954796118}

# Feature importance

In [54]:
all_features_used = TRANSFORMED_FEATURES \
    + CATEGORICAL_FEATURES \
    + [
        'count_surgery_diagnosis_codes',
        'count_surgery_internal_codes',
        'count_staff_main_surgeon_names',
        'count_surgeon_anes_names',
        'count_scrub_nurse_names',
        'count_circulating_nurse_names',
        'count_surgery_equipment_vendor_codes',
    ] \
    + NUMERICAL_FEATURES

In [55]:
feature_importances = tuned_xgboost['xgboost'].feature_importances_

In [56]:
assert(len(all_features_used)==len(feature_importances))

In [57]:
ranked_fi = [(feature, round(importance * 100, 2)) for feature, importance in zip(all_features_used, feature_importances)]

In [58]:
sorted(ranked_fi, key=lambda t:t[1], reverse=True)

[('or_reservation_operation_duration', 50.39),
 ('on_call', 13.65),
 ('surgery_internal_code', 7.68),
 ('scrub_nurse_name2', 6.92),
 ('count_circulating_nurse_names', 3.7),
 ('surgery_anes_method_code', 3.29),
 ('scrub_nurse_name1', 2.68),
 ('count_surgery_internal_codes', 1.2),
 ('surgery_department_code', 1.02),
 ('count_scrub_nurse_names', 1.01),
 ('count_staff_main_surgeon_names', 0.77),
 ('surgery_diagnosis_code', 0.73),
 ('staff_main_surgeon_name', 0.6),
 ('surgeon_anes_name4', 0.54),
 ('surgery_position_name', 0.52),
 ('surgery_classification_name', 0.49),
 ('count_surgeon_anes_names', 0.41),
 ('surgeon_anes_name5', 0.36),
 ('surgery_equipment_vendor_code2', 0.33),
 ('circulating_nurse_name2', 0.32),
 ('or_vendor_code', 0.3),
 ('surgeon_anes_name2', 0.3),
 ('surgery_position_code2', 0.3),
 ('circulating_nurse_name1', 0.27),
 ('surgeon_anes_name3', 0.26),
 ('surgery_ward_code', 0.26),
 ('count_surgery_equipment_vendor_codes', 0.25),
 ('year', 0.22),
 ('surgeon_anes_name1', 0.21),

# QA

## Test ImpactEncodingTransformer

In [59]:
ie = ImpactEncodingTransformer()

In [60]:
ie.fit(X_train[['surgery_classification_name', 'surgery_department_code']], y_train)

ImpactEncodingTransformer()

In [61]:
ie.transform(X_test)

,surgery_classification_name,surgery_department_code
9247,150,66
2733,218,271
8093,218,301
2966,218,264
3660,218,264
...,...,...
14863,218,21
10011,218,264
13392,218,520
5660,218,189


In [62]:
ie.impact_mapping;

## Test CountTransformer

In [63]:
temp = X[['surgery_diagnosis_code', 
    'surgery_diagnosis_code2', 
    'surgery_diagnosis_code3', 
    'surgery_diagnosis_code4', 
    'surgery_diagnosis_code5',]]

In [64]:
CountTransformer().fit_transform(temp);

## Test pipeline

In [65]:
transformation = Pipeline(
    steps=[
        ('select_initial_features', FeatureSelectorTransformer(INITIAL_FEATURES)),
        ('add_dow_transformer', AddDowTransformer()),
        ('select_more_features', FeatureSelectorTransformer(TRANSFORMED_FEATURES + INITIAL_FEATURES)),
        ('combine_all_transformers', combine_all_transformers),
    ]
)

In [66]:
transformation.fit(X_train, y_train)

Pipeline(steps=[('select_initial_features',
                 FeatureSelectorTransformer(features=['dow',
                                                      'surgery_classification_name',
                                                      'surgery_department_code',
                                                      'surgery_diagnosis_code',
                                                      'surgery_diagnosis_code2',
                                                      'surgery_diagnosis_code3',
                                                      'surgery_diagnosis_code4',
                                                      'surgery_diagnosis_code5',
                                                      'surgery_internal_code',
                                                      'surgery_internal_code2',
                                                      'surgery_internal_code3',
                                                      'surg...
                                                  CountTransformer(),
                                                  ['surgery_equipment_vendor_code1',
                                                   'surgery_equipment_vendor_code2',
                                                   'surgery_equipment_vendor_code3',
                                                   'surgery_equipment_vendor_code4',
                                                   'surgery_equipment_vendor_code5',
                                                   'surgery_equipment_vendor_code6',
                                                   'surgery_equipment_vendor_code7',
                                                   'surgery_equipment_vendor_code8']),
                                                 ('passthrough', 'passthrough',
                                                  ['or_reservation_operation_duration'])]))])

In [67]:
transformation.transform(X_test)

array([[166, 150,  66, ...,   1,   0,  30],
       [279, 218, 271, ...,   1,   0,  90],
       [279, 218, 301, ...,   1,   1, 180],
       ...,
       [222, 218, 520, ...,   1,   4, 370],
       [298, 218, 189, ...,   1,   3, 240],
       [166, 218,  66, ...,   2,   0,  45]], dtype=int64)

## Test AddDowTransformer

In [68]:
AddDowTransformer().fit_transform(X_train, y_train);